<a href="https://colab.research.google.com/github/poketmonster/sentimentanalysis/blob/master/ALC_SentimentAnalisys_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import csv, operator
import numpy as np
import nltk
import string
import re
import math
import random
import xml.etree.ElementTree as et
import pandas as pd
import time

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from random import shuffle
from nltk.stem.porter import *
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation


from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
!rm -r *
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_development.xml
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_test.xml
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_training.xml
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/ElhPolar_esV1.lex

--2019-04-14 10:40:48--  https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_development.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 163475 (160K) [text/plain]
Saving to: ‘TASS2017_T1_development.xml’

TASS2017_T1_develop 100%[===================>] 159.64K  --.-KB/s    in 0.03s   

2019-04-14 10:40:48 (4.98 MB/s) - ‘TASS2017_T1_development.xml’ saved [163475/163475]

--2019-04-14 10:40:50--  https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_test.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiti

In [24]:
#Cargar diccionario de polaridad

v = DictVectorizer(sparse=False)
s = open('ElhPolar_esV1.lex', 'r').read()


palabra = []
polaridad = []
with open('ElhPolar_esV1.lex') as f: 
  for line in f.readlines():
    div = line.split('\t')
    if len(div)==2:
      palabra.append(div[0])
      polaridad.append(div[1].replace("\n", ""))
  
Poldict = {}
for i in range(len(palabra)):
    Poldict[palabra[i]] = polaridad[i]
    
print(len(Poldict))
Poldict['abdicar']

4673


'negative'

In [0]:
def parse_XML(xml_file, df_cols): 
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    out_df = pd.DataFrame(columns = df_cols)
    
    for node in xroot: 
        res = []
        for el in df_cols[0:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index = True)
        
    return out_df

In [26]:
#Parse de los ficheros xml

train = parse_XML("TASS2017_T1_training.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print("TRAIN")
train.columns = ["tweetid", "user", "content", "date", "lang", "polarity"]
#print(train.head())
print("Langs: "+str(train.lang.unique()))
print(train['polarity'].value_counts(dropna=False))

dev = parse_XML("TASS2017_T1_development.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print("DEV")
dev.columns = ["tweetid", "user", "content", "date", "lang", "polarity"]
#print(dev.head())
print("Langs: "+str(dev.lang.unique()))
print(dev['polarity'].value_counts(dropna=False))

test = parse_XML("TASS2017_T1_test.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print("TEST")
test.columns = ["tweetid", "user", "content", "date", "lang", "polarity"]
#print(test.head())
print("Langs: "+str(test.lang.unique()))


TRAIN
Langs: ['es']
N       418
P       318
NONE    139
NEU     133
Name: polarity, dtype: int64
DEV
Langs: ['es']
N       219
P       156
NEU      69
NONE     62
Name: polarity, dtype: int64
TEST
Langs: ['es']


In [71]:
#Preparación de los listados de tweets i etiquetas

trainCorpus = []
train_labels = []

devCorpus = []
dev_labels = []

for i in range(0,len(train)):
  trainCorpus.append(train.at[i, 'content'])
  train_labels.append(train.at[i,'polarity'])

for i in range(0,len(dev)):
  devCorpus.append(dev.at[i, 'content'])
  dev_labels.append(dev.at[i,'polarity'])
  
print(len(trainCorpus))
print(len(train_labels))
print(len(devCorpus))
print(len(dev_labels))

#devCorpus

1008
1008
506
506


In [40]:
#Tokenizador propio

spanish_stopwords = stopwords.words('spanish')
print(spanish_stopwords)

non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))


tknzr = TweetTokenizer()

stemmer = SnowballStemmer('spanish')


def onwtokenizer(text):
  text = re.sub(r"http\S+", "", text)
  text = ''.join([c for c in text if c not in non_words])
  
  
  tokens = tknzr.tokenize(text)
  tokens = [w for w in tokens if not w in spanish_stopwords] 
  
  return tokens

#test
tokens = onwtokenizer("Hola @manolete como estas??? es ha ahora #colr https://www.google.com")

tokens

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros', 'vosostras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 

['Hola', 'manolete', 'ahora', 'colr']

In [19]:
# TfidfVectorizer CHAR
vectorizer = TfidfVectorizer(min_df=1,
                             max_df = 0.9,
                             sublinear_tf=True,
                             analyzer='char_wb',
                             ngram_range = (2, 9),
                             max_features = None,
                             strip_accents = "unicode",
                             use_idf=True)


train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors[1])

  (0, 70355)	0.02632714824595061
  (0, 45086)	0.01664526449100622
  (0, 70356)	0.019966355637173545
  (0, 50871)	0.01904063919418394
  (0, 12397)	0.021138641314692147
  (0, 6276)	0.021251354948307898
  (0, 48292)	0.011599288904551388
  (0, 15430)	0.024892681554804154
  (0, 86126)	0.02308825819489108
  (0, 101138)	0.01992722171358889
  (0, 15442)	0.025365509791046945
  (0, 86127)	0.02426634902353334
  (0, 101139)	0.01489304416367401
  (0, 15443)	0.026010037806274967
  (0, 86128)	0.015052515695187382
  (0, 15444)	0.015956168745707523
  (0, 11599)	0.02747187049190297
  (0, 33446)	0.014040701178467398
  (0, 69349)	0.016744634564403473
  (0, 12399)	0.022716117843325737
  (0, 14071)	0.011744841164714321
  (0, 42335)	0.017979167050127608
  (0, 5159)	0.02260726056667187
  (0, 30592)	0.01556819037043853
  (0, 65102)	0.027253151750489056
  :	:
  (0, 30324)	0.06414914131501313
  (0, 11600)	0.03560170120161268
  (0, 63485)	0.07919405770641294
  (0, 30331)	0.07355950177910446
  (0, 11611)	0.0382341

In [182]:
# TfidfVectorizer WORDS
vectorizer = TfidfVectorizer(min_df=1,
                             max_df = 0.99,
                             sublinear_tf=False,
                             analyzer='word',
                             ngram_range = (1,1),
                             max_features = None,
                             strip_accents = "ascii",
                             lowercase = True,
                             smooth_idf = True,
                             use_idf=True)

train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)

print(vectorizer.get_feature_names())
print(train_vectors.shape)

print(train_vectors[1])

['000', '02', '07', '10', '100', '10000', '1080', '10h', '11', '12', '14', '1477', '15', '16', '17', '19', '1994', '1a', '1r', '20', '2013', '2015', '2016', '2017', '2018', '22', '2294', '24', '24horas', '25', '25d', '27', '2a', '2bsrmwnrz5', '2gb', '2h', '2k17', '2kespana', '2t0s6dgxnm', '30', '300', '31', '31m', '32', '34', '36', '39', '3ds', '4a', '4k', '50', '500', '5875', '6239', '650', '69', '70', '72', '765', '7o', '88', '8chooficial', '8hlhyxox55', '8x4dvafgsp', '9qik0givzo', '__beatriizzzz', '__mxty', '_aitor', '_aliciagonzg', '_anamilan_', '_artificialred_', '_betterton', '_cuteresa', '_girlpride_', '_jesuismarc', '_raquel97_', '_tontonina', '_xoshiro', 'a7x', 'aaahm', 'aaron_np', 'abacos', 'abajo', 'abandonado', 'abandone', 'abatido', 'abcesos', 'aberron', 'abierta', 'abiertas', 'abierto', 'abiertos', 'abrazaros', 'abrazarte', 'abrazo', 'abrazos', 'abrazoteee', 'abre', 'abres', 'abrigo', 'abrigos', 'abrilnapilotti', 'abrir', 'absoluta', 'absolutamente', 'abstracto', 'absurda

In [8]:
# HashingVectorizer WORD
vectorizer = HashingVectorizer(n_features= 2 ** 18,
                             analyzer='word',
                             ngram_range = (1,1),
                             strip_accents = "ascii",
                             lowercase = True)

train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors.shape)

print(train_vectors[1])

(1008, 262144)
  (0, 12360)	0.25
  (0, 12716)	0.25
  (0, 28616)	-0.25
  (0, 36493)	0.25
  (0, 57122)	-0.25
  (0, 57778)	0.25
  (0, 92841)	0.25
  (0, 108449)	0.25
  (0, 118328)	-0.25
  (0, 120731)	0.25
  (0, 162710)	0.25
  (0, 172324)	-0.25
  (0, 203536)	0.25
  (0, 215326)	-0.25
  (0, 240216)	-0.25
  (0, 252348)	0.25


In [17]:
# HashingVectorizer CHAR
vectorizer = HashingVectorizer(n_features= 2 ** 18,
                             analyzer='char_wb',
                             ngram_range = (2,9),
                             strip_accents = "ascii",
                             lowercase = True)

train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors.shape)

print(train_vectors[1])

(1008, 262144)
  (0, 344)	0.04159451654038515
  (0, 1465)	0.04159451654038515
  (0, 1474)	-0.04159451654038515
  (0, 4203)	0.04159451654038515
  (0, 4211)	-0.04159451654038515
  (0, 4697)	0.04159451654038515
  (0, 4700)	0.04159451654038515
  (0, 5298)	0.0831890330807703
  (0, 5468)	0.04159451654038515
  (0, 6324)	0.04159451654038515
  (0, 7729)	0.04159451654038515
  (0, 7793)	-0.04159451654038515
  (0, 7841)	0.0831890330807703
  (0, 9036)	0.04159451654038515
  (0, 9661)	-0.04159451654038515
  (0, 10604)	-0.04159451654038515
  (0, 11389)	0.1663780661615406
  (0, 11764)	-0.04159451654038515
  (0, 12136)	-0.04159451654038515
  (0, 12360)	0.0831890330807703
  (0, 12716)	0.04159451654038515
  (0, 13009)	0.04159451654038515
  (0, 13348)	-0.2495670992423109
  (0, 13655)	-0.04159451654038515
  (0, 14246)	0.04159451654038515
  :	:
  (0, 244772)	-0.04159451654038515
  (0, 245130)	-0.04159451654038515
  (0, 245248)	-0.04159451654038515
  (0, 245530)	0.04159451654038515
  (0, 246034)	-0.0831890330

In [100]:
# CountVectorizer Char
vectorizer = CountVectorizer(min_df=1,
                             max_df = 0.9,
                             analyzer='char',
                             ngram_range = (2,6),
                             max_features = 50000,
                             strip_accents = "ascii",
                             lowercase = True)



train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors.shape)

print(train_vectors[1])

(1008, 50000)
  (0, 8163)	1
  (0, 19978)	2
  (0, 2356)	1
  (0, 33344)	1
  (0, 23523)	1
  (0, 35662)	1
  (0, 11372)	1
  (0, 32134)	1
  (0, 39678)	1
  (0, 9322)	1
  (0, 38795)	1
  (0, 3632)	1
  (0, 32865)	1
  (0, 14170)	1
  (0, 44316)	1
  (0, 38099)	1
  (0, 3606)	1
  (0, 7158)	1
  (0, 17461)	1
  (0, 2090)	1
  (0, 32370)	1
  (0, 28069)	1
  (0, 8776)	1
  (0, 38734)	1
  (0, 39960)	1
  :	:
  (0, 8593)	1
  (0, 17131)	1
  (0, 18842)	1
  (0, 14454)	3
  (0, 42122)	1
  (0, 3617)	1
  (0, 11270)	1
  (0, 21532)	1
  (0, 20305)	2
  (0, 8205)	1
  (0, 370)	2
  (0, 11800)	1
  (0, 3250)	1
  (0, 22111)	1
  (0, 9037)	1
  (0, 2351)	1
  (0, 44101)	2
  (0, 37992)	2
  (0, 3563)	2
  (0, 14829)	1
  (0, 766)	1
  (0, 2697)	2
  (0, 15946)	2
  (0, 12946)	2
  (0, 22958)	2


In [21]:
# CountVectorizer Word
vectorizer = CountVectorizer(min_df=1,
                             max_df = 0.7,
                             analyzer='word',
                             ngram_range = (1,1),
                             max_features = 50000,
                             strip_accents = "ascii",
                             lowercase = True)


train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors.shape[1])

print(train_vectors[1])

4791
  (0, 2307)	1
  (0, 197)	1
  (0, 880)	1
  (0, 1405)	1
  (0, 3751)	1
  (0, 3691)	1
  (0, 1986)	1
  (0, 2068)	1
  (0, 4265)	1
  (0, 4094)	1
  (0, 1548)	1
  (0, 2693)	1
  (0, 3682)	1
  (0, 3010)	1
  (0, 3688)	1
  (0, 2796)	1


In [44]:
#Diccinario de polaridad, es llama desde los pipelines pasándole el listado de tweets
def dict_tweet (tweets):
  res = []
  for tweet in tweets:
    neg = 0;
    pos = 0;

    #print (tweet)
    words = tweet.split(' ')
    for word in words:
      word = word.replace('-', '')
      word = word.replace('\n', '')
      word = word.replace('.', '')
      word = word.replace(',', '')
      word = word.replace('!', '')
      word = word.replace('""', '')
      word = word.replace('#', '')
      word = word.replace('*', '')
      word = word.replace('@', '')
      word = word.lower()
      if word in Poldict:
        if Poldict[word] == "negative":
          neg += 1
        if Poldict[word] == "positive":
           pos += 1
    total=neg+pos
    #res.append({pos-neg})
    
    if(total>0):
      res.append({'p' : pos-neg})
    else:
      res.append({'p' : 0})
    
  return res

#test
res = dict_tweet(['-Me caes muy bien \n-Tienes que jugar más partidas al lol con Russel y conmigo\n-Por qué tan Otako, deja de ser otako\n-Haber si me muero', 
                  '@myendlesshazza a. que puto mal escribo\n\nb. me sigo surrando help \n\n3. ha quedado raro el "cómetelo" ahí JAJAJAJA',
                  '@estherct209 jajajaja la tuya y la d mucha gente seguro!! Pero yo no puedo sin mi melena me muero '])

print(res[0]['p'])


1


In [45]:
#Test resultados Diccionario de polaridad

res = dict_tweet(trainCorpus)

correcto = 0
incorrecto = 0
neutral = 0
for i in range(0, len(trainCorpus)):
  #print(res[i])
  #print(train_labels[i])
  
  if res[i]['p'] < 0 and train_labels[i]=="N":
     correcto += 1
  elif res[i]['p'] > 0 and train_labels[i]=="P":
    correcto += 1
  elif res[i]['p'] == 0:
    neutral +=1
  else:
    incorrecto += 1
    
print ("Correcto "+str(correcto))
print ("Incorrecto "+str(incorrecto))
print ("Neutral "+str(neutral))


Correcto 401
Incorrecto 261
Neutral 346


In [0]:
class TextStats(BaseEstimator, TransformerMixin):
  
    def fit(self, x, y=None):
        return self

    def transform(self, tweet):
        return dict_tweet(tweet)

In [72]:
#PIPELINE TfidfVectorizer con diccionario

pipelineTfidfVectorizer =  Pipeline([

    # Unión de las features
    ('union', FeatureUnion(
        transformer_list=[

            # Vectorización de las palabras
            ('ngram', Pipeline([
                ('vect', TfidfVectorizer(min_df=1,
                             max_df = 0.9,
                             
                             sublinear_tf=False,
                             analyzer='word',
                             ngram_range = (1,1),
                             max_features = None,
                             strip_accents = "ascii",
                             lowercase = True,
                             smooth_idf = True,
                             use_idf=False)),
                ('tfidf', TfidfTransformer(norm='l2', 
                                           use_idf=True,
                                           smooth_idf=True,
                                           sublinear_tf=False)),
            ])),


            # Diccionario de polaridad
            ('diccionari', Pipeline([
                ('stats', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
            ])),

        ],

        # Pesos de cada parte
        transformer_weights={
            'subjngramect': 1,
            'diccionari': 0,
        },
    )),

    # Clasificador
    ('clf', svm.LinearSVC(penalty='l2',
                          loss='squared_hinge',
                          dual=True,
                          tol=0.0001, 
                          C=1.0, 
                          multi_class='ovr',
                          fit_intercept=True,
                          intercept_scaling=1,
                          class_weight=None,
                          verbose=0,
                          random_state=None,
                          max_iter=1000)),
])


#print (text_clf.named_steps['union'].transformer_list[0][1].named_steps['vect'].get_feature_names())


pipelineTfidfVectorizer.fit(trainCorpus, train_labels)

predicted = pipelineTfidfVectorizer.predict(devCorpus)


print(classification_report(dev_labels, predicted))


              precision    recall  f1-score   support

           N       0.59      0.82      0.69       219
         NEU       0.33      0.12      0.17        69
        NONE       0.42      0.18      0.25        62
           P       0.60      0.59      0.59       156

   micro avg       0.57      0.57      0.57       506
   macro avg       0.49      0.43      0.43       506
weighted avg       0.54      0.57      0.53       506



In [73]:
#PIPELINE CountVectorizer con diccionario

pipelineCountVectorizer =  Pipeline([

    # Unión de las features
    ('union', FeatureUnion(
        transformer_list=[

            # Vectorización de las palabras
            ('ngram', Pipeline([
                ('vect', CountVectorizer(min_df=1,
                             max_df = 0.9,
                             analyzer='char',
                             ngram_range = (2,6),
                             max_features = 50000,
                             strip_accents = "ascii",
                             lowercase = True)),
                ('tfidf', TfidfTransformer(norm=None, 
                                           use_idf=True,
                                           smooth_idf=True,
                                           sublinear_tf=False)),
            ])),


            # Diccionario de polaridad
            ('diccionari', Pipeline([
                ('stats', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
            ])),

        ],

        # Pesos de cada parte
        transformer_weights={
            'subjngramect': 1,
            'diccionari': 0.5,
        },
    )),

    # Clasificador
    ('clf', svm.LinearSVC(penalty='l2',
                          loss='squared_hinge',
                          dual=True,
                          tol=0.0001, 
                          C=1.0, 
                          multi_class='ovr',
                          fit_intercept=True,
                          intercept_scaling=1,
                          class_weight=None,
                          verbose=0,
                          random_state=None,
                          max_iter=10000)),
])


#print (text_clf.named_steps['union'].transformer_list[0][1].named_steps['vect'].get_feature_names())


pipelineCountVectorizer.fit(trainCorpus, train_labels)

predicted = pipelineCountVectorizer.predict(devCorpus)


print(classification_report(dev_labels, predicted))


              precision    recall  f1-score   support

           N       0.61      0.78      0.69       219
         NEU       0.23      0.13      0.17        69
        NONE       0.44      0.19      0.27        62
           P       0.62      0.64      0.63       156

   micro avg       0.58      0.58      0.58       506
   macro avg       0.48      0.44      0.44       506
weighted avg       0.54      0.58      0.55       506



In [68]:
    #SVM, kernel=rbf
    classifier_rbf = svm.SVC()
    classifier_rbf.fit(train_vectors, train_labels)
    prediction_rbf = classifier_rbf.predict(dev_vectors)

    #SVM, kernel=linear
    classifier_linear = svm.SVC(kernel='linear')
    classifier_linear.fit(train_vectors, train_labels)
    prediction_linear = classifier_linear.predict(dev_vectors)

    #SVM, kernel=linear
    classifier_liblinear = svm.LinearSVC()
    classifier_liblinear.fit(train_vectors, train_labels)
    prediction_liblinear = classifier_liblinear.predict(dev_vectors)

    #GaussianNB
    dense_train = train_vectors.toarray()
    dense_dev = dev_vectors.toarray()
    classifier_nv = GaussianNB()
    classifier_nv.fit(dense_train, train_labels)
    prediction_classifier_nv = classifier_nv.predict(dense_dev)
    
    #MultinomialNB
    dense_train = train_vectors.toarray()
    dense_dev = dev_vectors.toarray()
    classifier_mnv = MultinomialNB()
    classifier_mnv.fit(dense_train, train_labels)
    prediction_classifier_mnv = classifier_mnv.predict(dense_dev)
    
    #BernoulliNB
    dense_train = train_vectors.toarray()
    dense_dev = dev_vectors.toarray()
    classifier_bnv = BernoulliNB()
    classifier_bnv.fit(dense_train, train_labels)
    prediction_classifier_bnv = classifier_bnv.predict(dense_dev)
    
  
    # Print results in a nice table
    print("Clasificador SVC(kernel=rbf)")
    print(classification_report(dev_labels, prediction_rbf))
    
    print("Clasificador SVC(kernel=linear)")
    print(classification_report(dev_labels, prediction_linear))
    
    print("Clasificador LinearSVC()")
    print(classification_report(dev_labels, prediction_liblinear))
    
    print("Clasificador GaussianNB()")
    print(classification_report(dev_labels, prediction_classifier_nv))
    
    print("Clasificador MultinomialNB()")
    print(classification_report(dev_labels, prediction_classifier_mnv))
    
    print("Clasificador BernoulliNB()")
    print(classification_report(dev_labels, prediction_classifier_bnv))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Clasificador SVC(kernel=rbf)
              precision    recall  f1-score   support

           N       0.43      1.00      0.60       219
         NEU       0.00      0.00      0.00        69
        NONE       0.00      0.00      0.00        62
           P       0.00      0.00      0.00       156

   micro avg       0.43      0.43      0.43       506
   macro avg       0.11      0.25      0.15       506
weighted avg       0.19      0.43      0.26       506

Clasificador SVC(kernel=linear)
              precision    recall  f1-score   support

           N       0.54      0.92      0.68       219
         NEU       0.00      0.00      0.00        69
        NONE       0.00      0.00      0.00        62
           P       0.63      0.55      0.59       156

   micro avg       0.57      0.57      0.57       506
   macro avg       0.29      0.37      0.32       506
weighted avg       0.43      0.57      0.48       506

Clasificador LinearSVC()
              precision    recall  f1-score 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:
#Preparar corpus para ejecutar con test

trainCorpusFinal = []
train_labelsFinal = []

testCorpus = []

for i in range(0,len(train)):
  trainCorpusFinal.append(train.at[i, 'content'])
  train_labelsFinal.append(train.at[i,'polarity'])

for i in range(0,len(dev)):
  trainCorpusFinal.append(dev.at[i, 'content'])
  train_labelsFinal.append(dev.at[i,'polarity'])
  
for i in range(0, len(test)):
  testCorpus.append(test.at[i, 'content'])
  
print(len(trainCorpusFinal))
print(len(train_labelsFinal))
print(len(testCorpus))


1514
1514
1899


In [0]:
pipelineCountVectorizer.fit(trainCorpusFinal, train_labelsFinal)

predicted = pipelineCountVectorizer.predict(testCorpus)

In [76]:
file = open("RafaSanchis_LinearSVC.txt","w") 

for i in range(0, len(test)):
  file.write(test.at[i, 'tweetid']+"\t"+predicted[i]+"\n") 
  print(test.at[i, 'tweetid']+"\t"+predicted[i])

770567971701940224	P
770503386789711872	N
770502863017635840	P
770599972102348800	N
770599962216390656	N
770599405816864768	N
770453393714348032	NEU
770466143098241025	N
770536258682617856	NEU
770613199200620545	P
770612909105905665	N
770610067146235904	P
770633854667386880	P
770633713822601216	NONE
770443909260570624	P
770434045469659136	NEU
770463923342540800	N
770496679632076800	P
770456170691780608	N
770499448438915072	N
770510704080281600	P
770434546168897536	P
770081000747573248	P
770080539130916864	N
770080368636592129	N
770772755637764096	N
770789027142328326	P
770792054519656448	N
770773957775593472	N
770877057765105664	P
770942085960724480	NONE
770941295883853824	P
770940641601781760	P
770940052557950976	N
770948285288841216	P
770947728792817665	P
770947357173243906	NEU
770946843383631873	NEU
770943381652504576	N
770918556825427969	P
770918502865702912	P
770914817360195601	N
770914042567327746	N
770960812785692672	N
770960332818935809	P
770963910707019776	N
770963684705308672